# La motxilla

El problema de la motxilla és un dels problemes més coneguts en l'àmbit de la computació. El problema consisteix en omplir una motxilla amb objectes de diferent pes i valor, de manera que es maximitzi el valor de la motxilla sense excedir el pes màxim que pot suportar.

Per exemple, suposem que tenim una motxilla amb una capacitat de 100 kg i els següents objectes:

| Objecte   | Pes (kg) | Valor (€) |
|-----------|----------|-----------|
| Coca-Cola | 1        | 1.5       |
| Cervesa   | 2        | 2.5       |
| Aigua     | 1        | 0.5       |
| Cacauets  | 1        | 1.0       |
| Patates   | 2        | 1.5       |
| Galetes   | 1        | 1.0       |
| Pomes     | 1        | 1.0       |
| Taronges  | 1        | 1.0       |
| Platans   | 1        | 1.0       |
| Peres     | 1        | 1.0       |
| Maduixes  | 1        | 1.0       |
| Pinya     | 1        | 1.0       |
| Pomes     | 1        | 1.0       |

Quina combinació d'objectes podem posar a la motxilla per maximitzar el valor sense excedir el pes màxim?

En primer lloc necessitem definir les classes que ens permetran representar el problema, `Individu`, i `Genetic`. Una vegada definides les utilitzarem per resoldre el problema.

In [61]:
import random
from dataclasses import dataclass


@dataclass
class Individu:
    gens: list
    fitness: float = 0.0


class Genetic:
    def __init__(self, mida_poblacio, num_gens, valor_minim, valor_maxim, valors_discrets, funcio_fitness, prob_creuament, prob_mutacio, num_mutacions=1):

        self.mida_poblacio = mida_poblacio  # La mida de la població.
        self.num_gens = num_gens  # Número de gens per individu.
        self.valor_minim = valor_minim  # El valor mínim que pot tenir un gen.
        self.valor_maxim = valor_maxim  # El valor màxim que pot tenir un gen.
        self.valors_discrets = valors_discrets  # Booleà per determinar si els valors són discrets o no.
        self.funcio_fitness = funcio_fitness  # Funció per calcular el nivell d'adaptació d'un individu.
        self.prob_creuament = prob_creuament  # Probabilitat que es produeixi un creuament.
        self.prob_mutacio = prob_mutacio  # Probabilitat que es produeixi una mutació.
        self.num_mutacions = num_mutacions  # Número de mutacions a realitzar en cada generació.
        self.poblacio = list()  # Llista per emmagatzemar la població.
        self.generar_poblacio()  # Mètode per generar la població inicial.

    def generar_poblacio(self):
        """Genera la població inicial."""
        for _ in range(self.mida_poblacio):
            gens = self.generar_individu()
            fitness = self.funcio_fitness(gens)
            individu = Individu(gens=gens, fitness=fitness)
            self.poblacio.append(individu)

    def generar_individu(self):
        """Genera un individu aleatori."""
        individu = list()
        for _ in range(self.num_gens):
            individu.append(self.generar_gen())
        return individu

    def generar_gen(self):
        """Genera un gen aleatori."""
        if self.valors_discrets:
            return random.randint(self.valor_minim, self.valor_maxim)
        else:
            return random.uniform(self.valor_minim, self.valor_maxim)

    def ordenar_poblacio(self):
        """Ordena la població segons el seu nivell d'adaptació."""
        self.poblacio.sort(key=lambda x: x.fitness, reverse=True)

    def seleccionar_parella(self):
        """Selecciona una parella d'individus de la població. Utilitzarem el mètode del torneig."""
        parella = list()
        for _ in range(2):
            individus = random.sample(self.poblacio, 2)
            if individus[0].fitness > individus[1].fitness:
                parella.append(individus[0])
            else:
                parella.append(individus[1])
        return parella

    def creuar(self, parella):
        """Creua dos individus. Utilitzarem el mètode de creuament de 1 punt."""
        punt = random.randint(1, self.num_gens - 1)
        fill1 = parella[0].gens[:punt] + parella[1].gens[punt:]
        fill2 = parella[1].gens[:punt] + parella[0].gens[punt:]
        fitness_fill1 = self.funcio_fitness(fill1)
        fitness_fill2 = self.funcio_fitness(fill2)
        return (Individu(gens=fill1, fitness=fitness_fill1),
                Individu(gens=fill2, fitness=fitness_fill2))

    def mutar(self, fill):
        """Muta un individu. Utilitzarem el mètode de mutació uniforme."""
        for _ in range(self.num_mutacions):
            gen = random.randint(0, self.num_gens - 1)
            fill.gens[gen] = self.generar_gen()
        return fill

    def executar(self, num_generacions):
        """Executa l'algorisme genètic."""
        for i in range(num_generacions):
            self.ordenar_poblacio()

            print(f'Generació: {i}: {self.poblacio[0].fitness}')
            print(f'Individu: {self.poblacio[0].gens}')

            nova_poblacio = []

            while len(nova_poblacio) < self.mida_poblacio:
                parella = self.seleccionar_parella()
                if random.random() < self.prob_creuament:
                    fill1, fill2 = self.creuar(parella)
                    if random.random() < self.prob_mutacio:
                        fill1 = self.mutar(fill1)
                    if random.random() < self.prob_mutacio:
                        fill2 = self.mutar(fill2)
                else:
                    fill1, fill2 = parella

                nova_poblacio.append(fill1)
                nova_poblacio.append(fill2)

            self.poblacio = nova_poblacio

Ara que ja tenim les classes definides, podem començar a resoldre el problema. En primer lloc definirem les variables que necessitem per resoldre el problema.

In [62]:
PES_MAXIM = 100
PRODUCTES = [
    ('Coca-Cola', 1, 1.5),
    ('Cervesa', 2, 2.5),
    ('Aigua', 1, 0.5),
    ('Cacauets', 1, 1.0),
    ('Patates', 2, 1.5),
    ('Galetes', 1, 1.0),
    ('Pomes', 1, 1.0),
    ('Taronges', 1, 1.0),
    ('Platans', 1, 1.0),
    ('Peres', 1, 1.0),
    ('Maduixes', 1, 1.0),
    ('Pinya', 1, 1.0),
    ('Pomes', 1, 1.0)
]

Per fer-ho, primer necessitem definir la funció de fitness. En aquest cas, la funció de fitness serà la suma dels valors dels objectes que hi ha a la motxilla. A més, necessitem definir els gens que utilitzarem per representar cada objecte. En aquest cas, utilitzarem un gen per cada objecte, i el valor del gen serà el nombre d'objectes d'aquell tipus que hi ha a la motxilla. Per exemple, si el gen 0 té el valor 2, això vol dir que hi ha 2 cerveses a la motxilla. Així doncs, la funció de fitness serà:

In [63]:
def funcio_fitness(gens):
    """Funció de fitness."""
    valor = sum(gens[i] * PRODUCTES[i][2] for i in range(len(gens)))
    pes = sum(gens[i] * PRODUCTES[i][1] for i in range(len(gens)))
    
    exces = PES_MAXIM - pes
    if exces < 0:
        valor = valor + exces * 1000
    
    print(f'Valor: {valor}, Pes: {pes}')
    return valor
    

Ara que ja tenim la funció de fitness definida, podem crear l'algorisme genètic. En aquest cas, utilitzarem els següents paràmetres:

In [64]:
mida_poblacio = 100
num_gens = len(PRODUCTES)
valor_minim = 0
valor_maxim = 10
valors_discrets = True
prob_creuament = 0.8
prob_mutacio = 0.1
num_mutacions = 1

Ara ja podem crear l'algorisme genètic i executar-lo.

In [65]:
genetic = Genetic(mida_poblacio=mida_poblacio,
                  num_gens=num_gens,
                  valor_minim=valor_minim,
                  valor_maxim=valor_maxim,
                  valors_discrets=valors_discrets,
                  funcio_fitness=funcio_fitness,
                  prob_creuament=prob_creuament,
                  prob_mutacio=prob_mutacio,
                  num_mutacions=num_mutacions)
genetic.executar(num_generacions=100)

Valor: 82.5, Pes: 80
Valor: 85.5, Pes: 83
Valor: 40.5, Pes: 42
Valor: 78.0, Pes: 80
Valor: 82.0, Pes: 82
Valor: 87.5, Pes: 91
Valor: 83.0, Pes: 82
Valor: 80.0, Pes: 75
Valor: 85.0, Pes: 79
Valor: 65.0, Pes: 66
Valor: 77.0, Pes: 79
Valor: 68.5, Pes: 70
Valor: 83.5, Pes: 84
Valor: 69.5, Pes: 68
Valor: 77.0, Pes: 75
Valor: 51.0, Pes: 54
Valor: 48.0, Pes: 50
Valor: 90.0, Pes: 92
Valor: 56.0, Pes: 63
Valor: 78.5, Pes: 75
Valor: 85.5, Pes: 85
Valor: 90.5, Pes: 89
Valor: 80.5, Pes: 83
Valor: 105.5, Pes: 99
Valor: 77.0, Pes: 81
Valor: 81.0, Pes: 78
Valor: 64.0, Pes: 61
Valor: 57.5, Pes: 57
Valor: 69.5, Pes: 70
Valor: 67.0, Pes: 65
Valor: 86.5, Pes: 85
Valor: 52.5, Pes: 50
Valor: 89.5, Pes: 87
Valor: 61.0, Pes: 58
Valor: 56.0, Pes: 58
Valor: 66.0, Pes: 73
Valor: 78.0, Pes: 80
Valor: 86.0, Pes: 89
Valor: 80.0, Pes: 77
Valor: 62.5, Pes: 66
Valor: 52.5, Pes: 53
Valor: 76.0, Pes: 74
Valor: 57.0, Pes: 61
Valor: 74.0, Pes: 70
Valor: 79.0, Pes: 73
Valor: 93.0, Pes: 93
Valor: 54.0, Pes: 56
Valor: 79.0,

Una vegada executat l'algorisme genètic, podem veure quin és el millor individu que s'ha trobat.

In [66]:
genetic.ordenar_poblacio()
print(f'Valor: {genetic.poblacio[0].fitness}')
print(f'Pes: {sum(genetic.poblacio[0].gens[i] * PRODUCTES[i][1] for i in range(len(genetic.poblacio[0].gens)))}')
print(f'Individu: {genetic.poblacio[0].gens}')

Valor: 110.0
Pes: 100
Individu: [10, 10, 0, 10, 0, 7, 5, 9, 8, 7, 9, 7, 8]
